## Algoritmo apriori con mlxtend

In [3]:
#Importamos las librerias necesarias
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules #Importar la funcion apriori y association_rules
from mlxtend.preprocessing import TransactionEncoder #Importar la funcion TransactionEncoder para transformar los datos

#### Dataset de ejemplo (Transaccion de supermercado) Vamos a usar un pequeño dataset de ejemplo que representa compras en un supermercado

In [ ]:
#Cada lista interna representa una transaccion (canasta de compra)
transacciones = [
    ['leche', 'pan', 'manzanas'],
    ['leche', 'pan', 'huevos','yogur'],
    ['pan','queso', 'huevos'],
    ['leche','queso', 'huevos','manzanas'],
    ['pan','leche', 'queso', 'huevos'],
    ['pan','leche', 'queso']
]

print("Dataset original")
for i, trans in enumerate(transacciones, 1):
    print(f"Transacción {i}: {trans}")

Dataset original
Transacción 1: ['leche', 'pan', 'manzanas']
Transacción 2: ['leche', 'pan', 'huevos', 'yogur']
Transacción 3: ['pan', 'queso', 'huevos']
Transacción 4: ['leche', 'queso', 'huevos', 'manzanas']
Transacción 5: ['pan', 'leche', 'queso', 'huevos']
Transacción 6: ['pan', 'leche', 'queso']


#### Preprocesamiento de datos: Convertimos las transacciones a un formato one-hot(binario) que necesita apriori

In [4]:
#Inicializamos el codificador de transacciones
te = TransactionEncoder()

#Ajustamos y transformamos los datos a formato one-hot
#(cada producto se convierte en una columna y cada transacción en una fila)
#El resultado es una matriz booleana donde True indica la presencia del producto en la transacción
te_ary = te.fit(transacciones).transform(transacciones)

#Convertimos a DataFrame de pandaspara mejor visualizacion}
df = pd.DataFrame(te_ary, columns=te.columns_)
print("\nDataFrame transformado (One-Hot Encoding): ")
print(df)


DataFrame transformado (One-Hot Encoding): 
   huevos  leche  manzanas    pan  queso  yogur
0   False   True      True   True  False  False
1    True   True     False   True  False   True
2    True  False     False   True   True  False
3    True   True      True  False   True  False
4    True   True     False   True   True  False
5   False   True     False   True   True  False


#### Aplicacion del algoritmo apriori: Encontramos itemsets frecuentes (combinaciones de productos que aparecen juntos con frecuencia):

In [5]:
#Calculamos los itemsets frecuentes con un soporte mínimo del 50%
#min_support=0.5 significa que el itemset debe aparecer en al menos el 50% de las transacciones
# use_colnames=True significa que queremos los nombres de los productos en lugar de índices numericos
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)

print("\nItemsets frecuentes (soporte >= %50):")
print(frequent_itemsets.sort_values('support', ascending=False))


Itemsets frecuentes (soporte >= %50):
    support         itemsets
1  0.833333          (leche)
2  0.833333            (pan)
0  0.666667         (huevos)
3  0.666667          (queso)
7  0.666667     (pan, leche)
4  0.500000  (leche, huevos)
5  0.500000    (pan, huevos)
6  0.500000  (huevos, queso)
8  0.500000   (leche, queso)
9  0.500000     (pan, queso)


#### Generacion de reglas de asociacion: Ahora generamos reglas como "Si compran X, entonces compran Y"

In [6]:
#Generamos reglas con confianza mínima del 70%
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

#Filtramos reglas con lift > 1 (relaciones significativas)
rules = rules[rules['lift'] > 1]

print("\nReglas de asociación (confianza >= 70% y lift > 1):")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values('lift', ascending=False))



Reglas de asociación (confianza >= 70% y lift > 1):
  antecedents consequents  support  confidence   lift
2    (huevos)     (queso)      0.5        0.75  1.125
3     (queso)    (huevos)      0.5        0.75  1.125


In [ ]:
'''
Soporte (Support) = 0.5 (50%)
Qué significa: El conjunto {Huevos, Queso} aparece en el 50% de todas las transacciones.
Implicación: Es una combinación relativamente común en tus datos.

Confianza (Confidence) = 0.75 (75%)
Para {Huevos} → {Queso}: Cuando los clientes compran Huevos, el 75% de las veces también compran Queso.
Para {Queso} → {Huevos}: Cuando los clientes compran Queso, el 75% de las veces también compran Huevos.

Lift = 1.125
Un lift > 1 indica que existe una relación positiva entre los productos.
1.125 significa que la probabilidad de que compren ambos juntos es 1.125 veces mayor que si fueran independientes.
No es una relación extremadamente fuerte (lift > 1.5 sería más significativo), pero sí existe cierta asociación.
'''